In [85]:
from pyspark.sql import SparkSession, types, functions as F

In [148]:
spark = SparkSession\
  .builder\
  .master("local[*]")\
  .appName('test')\
  .config("spark.driver.bindAddress", "127.0.0.1")\
  .getOrCreate()

In [149]:
df_nested = spark.read\
  .option("multiline", "true")\
  .json("./data/matches/*/*")

In [ ]:
df_flat = df_nested.select(
    F.col('match_id').cast(types.IntegerType()),
    F.col('match_date').cast(types.DateType()),
    F.col('match_week').cast(types.IntegerType()),
    F.col('kick_off').cast(types.TimestampType()),
    F.col('season.season_id').cast(types.IntegerType()).alias('season_id'),
    F.col('season.season_name').cast(types.StringType()).alias('season_name'),
    F.col('competition.competition_id').cast(types.IntegerType()).alias('competition_id'),
    F.col('competition.competition_name').cast(types.StringType()).alias('competition_name'),
    F.col('competition.country_name').cast(types.StringType()).alias('competition_country'),
    F.col('competition_stage.id').cast(types.IntegerType()).alias('competition_stage_id'),
    F.col('competition_stage.name').cast(types.StringType()).alias('competition_stage_name'),
    F.col('home_team.home_team_id').cast(types.IntegerType()).alias('home_team_id'),
    F.col('home_team.home_team_name').cast(types.StringType()).alias('home_team_name'),
    F.col('home_team.home_team_gender').cast(types.StringType()).alias('home_team_gender'),
    F.col('home_team.home_team_group').cast(types.StringType()).alias('home_team_group'),
    F.col('home_team.country.id').cast(types.IntegerType()).alias('home_team_country_id'),
    F.col('home_team.country.name').cast(types.StringType()).alias('home_team_country'),
    F.col('away_team.away_team_id').cast(types.IntegerType()).alias('away_team_id'),
    F.col('away_team.away_team_name').cast(types.StringType()).alias('away_team_name'),
    F.col('away_team.away_team_gender').cast(types.StringType()).alias('away_team_gender'),
    F.col('away_team.away_team_group').cast(types.StringType()).alias('away_team_group'),
    F.col('away_team.country.id').cast(types.IntegerType()).alias('away_team_country_id'),
    F.col('away_team.country.name').cast(types.StringType()).alias('away_team_country'),
    F.col('home_score').cast(types.IntegerType()),
    F.col('away_score').cast(types.IntegerType()),
    F.col('referee.id').cast(types.IntegerType()).alias('referee_id'),
    F.col('referee.name').cast(types.StringType()).alias('referee_name'),
    F.col('stadium.id').cast(types.IntegerType()).alias('stadium_id'),
    F.col('stadium.name').cast(types.StringType()).alias('stadium_name'),
    F.col('stadium.country.id').cast(types.IntegerType()).alias('stadium_country_id'),
    F.col('stadium.country.name').cast(types.StringType()).alias('stadium_country_name')
)

In [155]:
df_flat.printSchema()

root
 |-- match_id: integer (nullable = true)
 |-- match_date: date (nullable = true)
 |-- match_week: integer (nullable = true)
 |-- kick_off: timestamp (nullable = true)
 |-- season_id: integer (nullable = true)
 |-- season_name: string (nullable = true)
 |-- competition_id: integer (nullable = true)
 |-- competition_name: string (nullable = true)
 |-- competition_country: string (nullable = true)
 |-- competition_stage_id: integer (nullable = true)
 |-- competition_stage_name: string (nullable = true)
 |-- home_team_id: integer (nullable = true)
 |-- home_team_name: string (nullable = true)
 |-- home_team_gender: string (nullable = true)
 |-- home_team_group: string (nullable = true)
 |-- home_team_country_id: integer (nullable = true)
 |-- home_team_country: string (nullable = true)
 |-- away_team_id: integer (nullable = true)
 |-- away_team_name: string (nullable = true)
 |-- away_team_gender: string (nullable = true)
 |-- away_team_group: string (nullable = true)
 |-- away_team_c

In [147]:
spark.stop()